In [82]:
#| default_exp chat

In [83]:
# import getpass
# import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [84]:
# import getpass
# import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-4o-mini")

In [85]:
#| export 
from langchain_core.messages import HumanMessage, SystemMessage

from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [86]:
#| export 
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

helperPrompt  = """You are a job search assistant. 
You are helping a job searcher find jobs to apply for. From the list of jobs you have, you need to help the job searcher find the jobs that are relevant to them. You can ask the job searcher questions to help you understand what they are looking for. 

The current requirements of that the user has provided will be provided to you with their latest message, along with the search results that the search engine has returned for those requirements. If the search results doesn't return any results let the user know. Do not refer to or discuss any job postings that aren't in the search results. 
"""

basePrompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            helperPrompt,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = basePrompt | model


In [87]:
requirementsPrompt  = """Current search requirements are: {
    "Job Title": "Software Engineer",
    "Location": "Remote",
    "Company": "Google"
}
"""

searchResultsPrompt = """
Current search results are: [
    {
        "Job Title": "Software Engineer",
        "Location": "Remote",
        "Company": "Google",
        "Status": "Pending"
    },
    {
        "Job Title": "Software Engineer",
        "Location": "Onsite",
        "Company": "Facebook",
        "Status": "Pending"
    },
    {
        "Job Title": "Software Engineer",
        "Location": "Remote",
        "Company": "Amazon",
        "Status": "Pending"
    }
]
"""

In [88]:
#| export
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory
)

from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")


In [89]:
#| export 
config = {"configurable": {"session_id": "abc4"}}

In [90]:
#| export
def remove_previous_RAG_info(session_id: str) -> dict:
    history = get_session_history(session_id)

    alteredHistory = []

    for message in history.messages:
        if type(message) is tuple:
            msgType, text = message
            if "Current search requirements are" not in text:
                if "Current search results are" not in text:
                    alteredHistory.append(message)    
        elif type(message) is HumanMessage:
            alteredHistory.append(message)
        else:
            alteredHistory.append(message)

    history.messages = alteredHistory

    return {"session_id": session_id, "messages": alteredHistory}

In [91]:
#| export
from langchain_core.prompts import ChatPromptTemplate

requirementsPrompt = ChatPromptTemplate.from_template(
"""Current search requirements are: {requirements}""")

def getRequirementsPrompt(requirements):
    return requirementsPrompt.format(requirements=requirements)


In [92]:
#| export
from langchain_core.prompts import ChatPromptTemplate

searchPrompt = ChatPromptTemplate.from_template(
"""Search results meeting those requirements are: {requirements}""")

def getSearchResultsPrompt(requirements):
    return searchPrompt.format(requirements=requirements)


In [93]:
#| export

def getChatResponse(input, requirements, searchResults):
    # Remove the previous RAG info, so we can re-run the bot with new RAG info
    remove_previous_RAG_info("abc4")

    requirementsPrompt = getRequirementsPrompt(requirements)
    searchResultsPrompt = getSearchResultsPrompt(searchResults)

    response = with_message_history.invoke(
        { "messages": [
            (   "system",
                requirementsPrompt
            ),
            (   "system",
                searchResultsPrompt
            ),
            HumanMessage(content=input)]
        },
        config=config,
    )
    
    return response.content

In [95]:
#| hide
from nbdev import nbdev_export
nbdev_export()